# GRU

*Gated Recurent Unit*

ゲート付き回帰型ユニット, ゲート付きRNN

RNN層にゲートと呼ばれる機構を追加して長期的な文脈が保持できるようになったもの。  
通常のRNNは長期的な文脈の保持が苦手とされている。

In [1]:
import os; os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import random
from typing import List

import tensorflow as tf
import tensorflow_datasets as tfds
import sentencepiece as spm
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from torchtext import transforms
from torchtext.vocab import build_vocab_from_iterator
from torchvision.transforms import Compose
from dlprog import train_progress


---

## ゲート

あるデータをどれくらい通すかを決める機構。0-1の値を出力する。  
NNで実装してみる。

In [2]:
class Gate(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_size, input_size),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.net(x)

入力されたデータを線形変化し、sigmoid関数に入力するだけ。  
これを元のデータに掛けることで、元のデータの一部を**通した**ということになる。

In [3]:
input_size = 3
gate = Gate(input_size)

x = torch.randn(input_size)
y = x * gate(x)
print('input:', x)
print('gate:', gate(x))
print('output:', y)

input: tensor([0.3494, 0.8851, 1.5004])
gate: tensor([0.3011, 0.5463, 0.2664], grad_fn=<SigmoidBackward0>)
output: tensor([0.1052, 0.4835, 0.3997], grad_fn=<MulBackward0>)



---

## GRUの構造

GRUの構造とその利点を見ていこう。

一旦RNNの復習をしよう。


RNNはある時間$t$の入力$x_t$に対して以下のような演算で出力値$h_t$を決定する。

$$
h_t = \mathrm{tanh}(W_x x_t + b_x + W_h h_{t-1} + b_h)
$$

この$x_t$と$h_{t-1}$の全結合の部分は$\mathrm{fc}(x,h)$で表そう。

$$
\begin{align}
h_t &= \mathrm{tanh}(\mathrm{fc}(x_t,h_{t-1})) \\
\mathrm{fc}(x,h) &= W_x x + b_x + W_h h + b_h
\end{align}
$$

$\mathrm{fc}(x,h)$の実装もしておこう。

In [4]:
class FullyConnected(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.fc_input = nn.Linear(input_size, output_size)
        self.fc_hidden = nn.Linear(output_size, output_size)

    def forward(self, x, h):
        return self.fc_input(x) + self.fc_hidden(h)

では、GRUの構造を見ていこう。  
GRUは以下のような演算で出力値$h_t$を決定する。

$$
\begin{align}
h_t &= (1 - z_t) \odot \tilde{h}_t + z_t \odot h_{t-1} \\
\tilde{h}_t &= \mathrm{tanh}(\mathrm{fc}_{\tilde h}(x_t,h_{t-1})) \\
z_t &= \sigma(\mathrm{fc}_{z}(x_t,h_{t-1})) \\
\end{align}
$$

$\sigma(x)$はsigmoid関数。

RNNでは新たなデータ$\tilde h_t$がそのまま出力されていた。  
GRUでは、新たなデータ$\tilde h_t$を古いデータ$h_{t-1}$に足して出力する。そして、その際の比率をゲート$z_t$で決める。この$z_t$は$h_{t-1}$をどれだけ通すかを表す。

In [5]:
class SimpleGRU(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.fc = FullyConnected(input_size, output_size)
        self.gate = nn.Sequential(
            FullyConnected(input_size, output_size),
            nn.Sigmoid()
        )

    def forward(self, x, h):
        h_new = F.tanh(self.fc(x, h))
        z = self.gate(x)
        h = (1 - z) * h_new + z * h
        return h

このように、GRUではゲートを用いて新たなデータをどれだけ取り入れるべきか、そして古いデータをどれだけ捨てるか考えることが出来る。  
この枠組みの下で学習を行うことで、長期的に保持すべきデータをしっかりと保持できるようになることが期待される。

ちなみに、上記のモデルは一般的なGRUよりも簡略化されている。  
一般的なGRUは、上記のモデルにゲートを一つ追加した以下の様なモデルである。


$$
\begin{align}
h_t &= (1 - z_t) \odot \tilde{h}_t + z_t \odot h_{t-1} \\
\tilde{h}_t &= \mathrm{tanh}(\mathrm{fc}_{\tilde h}(x_t,r_t \odot h_{t-1})) \\
z_t &= \sigma(\mathrm{fc}_{z}(x_t,h_{t-1})) \\
r_t &= \sigma(\mathrm{fc}_{r}(x_t,h_{t-1})) \\
\end{align}
$$

新はデータ$\tilde h_t$を生成する際に、古いデータ$h_{t-1}$をどれだけ考慮するかを決めるゲート$r_t$が追加されている。

In [6]:
class GRU(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.fc_input = FullyConnected(input_size, output_size)
        self.gate_update = nn.Sequential(
            FullyConnected(input_size, output_size),
            nn.Sigmoid()
        )
        self.gate_reset = nn.Sequential(
            FullyConnected(input_size, output_size),
            nn.Sigmoid()
        )

    def forward(self, x, h):
        r = self.gate_reset(x, h)
        h_new = F.tanh(self.fc_input(x, r * h))
        z = self.gate_update(x)
        h = (1 - z) * h_new + z * h
        return h

また、RNN同様、PyTorchにクラスとして`torch.nn.GRU`が用意されている:  
[GRU — PyTorch 2.0 documentation](https://pytorch.org/docs/stable/generated/torch.nn.GRU.html)

In [7]:
gru = nn.GRU(input_size, input_size)